In [ ]:
import torch
import datetime
import random
import time

import pandas as pd
import numpy as np

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

from torch.optim import AdamW
from torch.utils.data import RandomSampler, SequentialSampler, TensorDataset, DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup

from datasets import load_dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import datasets
print(datasets.__version__)

In [ ]:
#!pip install -U datasets --quiet

In [ ]:
# NLI 데이터 중 학습데이터(split="train")를 로드.
cs = load_dataset('klue', name = 'nli', split = 'train')

# NLI 데이터 중 테스트데이터(split="validation")을 로드.
test_cs = load_dataset('klue', name = 'nli', split = 'validation')

cs = cs.train_test_split(0.1, seed = 777)
train_cs = cs['train']
valid_cs = cs['test']

In [ ]:
train_cs[:2]

In [ ]:
train_sentence = list(map(lambda train_cs : '[CLS] ' + str(train_cs['premise']) + ' [SEP] ' + str(train_cs['hypothesis']) + ' [SEP]', train_cs))
valid_sentence = list(map(lambda valid_cs : '[CLS] ' + str(valid_cs['premise']) + ' [SEP] ' + str(valid_cs['hypothesis']) + ' [SEP]', valid_cs))
test_sentence = list(map(lambda test_cs : '[CLS] ' + str(test_cs['premise']) + ' [SEP] ' + str(test_cs['hypothesis']) + ' [SEP]', test_cs))

In [ ]:
train_labels = train_cs['label']
valid_labels = valid_cs['label']
test_labels = test_cs['label']

In [ ]:
valid_sentence[:2]

In [ ]:
valid_labels[:2]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
max_len = 128

def data_to_tensor(sentences, labels, max_len):

  # 토큰화
  tokenized_sentence = [tokenizer.tokenize(sent) for sent in sentences]

  # 정수 인코딩
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_sentence]

  # 패딩
  input_ids = pad_sequences(input_ids, maxlen = max_len, dtype = 'long', truncating = 'post', padding = 'post')

  # 어텐션 마스크
  attention_mask = []

  for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_mask.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_mask)

  return tensor_inputs, tensor_labels, tensor_masks

In [ ]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentence, train_labels, max_len)
valid_inputs, valid_labels, valid_masks = data_to_tensor(valid_sentence, valid_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentence, test_labels, max_len)

In [ ]:
print('정수인코딩결과:',test_inputs[0])
print('-'*20)
print('원본문장복원결과:', tokenizer.decode(test_inputs[0]))
print('-'*20)
print('어텐션마스크:', test_masks[0])
print('-'*20)
print('샘플의길이:', len(test_inputs[0]))
print('-'*20)
print('레이블:', test_labels[0])

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler = valid_sampler, batch_size = batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
num_labels = 3
model = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels = num_labels)
model.to(device)

In [ ]:
epochs = 3
optimizer = AdamW(model.parameters(), lr = 2e-5)

In [ ]:
def metrics(prediction, labels):
  y_pred = prediction
  y_true = labels

  accuracy = accuracy_score(y_pred = y_pred, y_true = y_true)
  f1_micro = f1_score(y_pred = y_pred, y_true = y_true, average = 'micro', zero_division = 0)
  f1_macro = f1_score(y_pred = y_pred, y_true = y_true, average = 'macro', zero_division = 0)
  f1_weight = f1_score(y_pred = y_pred, y_true = y_true, average = 'weighted', zero_division = 0)

  metrics = {'accuracy' : accuracy,
             'f1_micro' : f1_micro,
             'f1_macro' : f1_macro,
             'f1_weight' : f1_weight}

  return metrics

In [ ]:
def train_epoch(model, train_dataloader, optimzier, device):
  total_loss = 0
  model.train()

  for step, batch in tqdm(enumerate(train_dataloader), desc = 'Training Batch'):
    batch = tuple(t.to(device) for t in batch)
    b_ids, b_mask, b_labels = batch

    outputs = model(b_ids, token_type_ids = None, attention_mask = b_mask, labels = b_labels)
    loss = outputs.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  return total_loss / len(train_dataloader)

In [ ]:
def evaluation(model, valid_dataloader, device):
  total_loss = 0
  pred = []
  true = []

  model.eval()

  for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_ids, b_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_ids, token_type_ids = None, attention_mask = b_mask, labels = b_labels)

    if outputs.loss is not None:
      loss = outputs.loss
      total_loss += loss.item()

    logits = outputs.logits.detach().cpu().numpy()
    labels_ids = b_labels.to('cpu').numpy()

    pred.extend(np.argmax(logits, axis = 1).flatten())
    true.extend(labels_ids.flatten())

  eval_metrics = metrics(pred, true)

  return total_loss / len(valid_dataloader), eval_metrics

In [ ]:
min_val_loss = float('inf')

for epoch in range(epochs):
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
  train_epoch(model, train_dataloader, optimizer, device)

  print("\nRunning Validation...")
  valid_loss, eval_metrics  = evaluation(model, valid_dataloader, device)
  print(" Validation Loss: {0:.2f}".format(valid_loss))
  print(" Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
  print(" F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
  print(" F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
  print(" F1 Weighted: {0:.2f}".format(eval_metrics['f1_weight']))

  if valid_loss < min_val_loss:
    print(f"Validation loss decreased ({min_val_loss:.2f}--> {valid_loss:.2f}). Saving model ...")
    torch.save(model.state_dict(), 'best_model.pt')
    min_val_loss = valid_loss

In [ ]:
model.load_state_dict(torch.load('best_model.pt'))

valid_loss, eval_metrics = evaluation(model, test_dataloader, device)
print(" Test Loss: {0:.2f}".format(valid_loss))
print(" Accuracy:{0:.2f}".format(eval_metrics['accuracy']))
print(" F1 Macro:{0:.2f}".format(eval_metrics['f1_macro']))
print(" F1 Micro:{0:.2f}".format(eval_metrics['f1_micro']))
print(" F1 Weighted: {0:.2f}".format(eval_metrics['f1_weight']))

In [ ]:
from transformers import pipeline
pipe2 = pipeline('text-classification', model = model.to(device), tokenizer = tokenizer, device = 0, max_length = 512, return_all_scores = True, function_to_apply = 'softmax')
inputs2 = {"text" : "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.", "text_pair" : "어떤 방에서도 흡연은 금지됩니다."}
result2 = pipe2([inputs2])
print(result2)

In [ ]:
pipe = pipeline('text-classification', model = model.to(device), tokenizer = tokenizer, device = 0, max_length = 512, function_to_apply = 'softmax')

In [ ]:
label_dict={'LABEL_0' :'얽힘','LABEL_1': '중립', 'LABEL_2' : '모순'}

def prediction(sent1, sent2):
  text = {"text" : sent1,"text_pair" : sent2}
  result = pipe(text)
  return [label_dict[result['label']]]

In [ ]:
sent1 = "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다."
sent2 = "어떤 방에서도 흡연은 금지됩니다."

prediction(sent1, sent2)

In [ ]:
sent1 = "저는,그냥알아내려고거기있었어요."
sent2 = "나는돈이어디로갔는지이해하려고했어요."

prediction(sent1, sent2)

In [ ]:
sent1 = "저는 그것을 이해하려고 거기 있었어요."
sent2 = "저는 이해하려고 노력하고 있었어요."

prediction(sent1, sent2)